In [ ]:
!rm -r '513'

In [ ]:
!unzip -qq '/content/valid.zip'

In [ ]:
!zip -r 516.zip 516

  adding: 516/ (stored 0%)
  adding: 516/trip_49_part_1.csv (deflated 80%)
  adding: 516/.ipynb_checkpoints/ (stored 0%)
  adding: 516/trip_62_part_1.csv (deflated 80%)
  adding: 516/trip_90_part_2.csv (deflated 93%)
  adding: 516/trip_18_part_2.csv (deflated 79%)
  adding: 516/trip_50_part_1.csv (deflated 82%)
  adding: 516/trip_113_part_2.csv (deflated 82%)
  adding: 516/trip_50_part_2.csv (deflated 75%)
  adding: 516/trip_37_part_1.csv (deflated 80%)
  adding: 516/trip_83_part_2.csv (deflated 80%)
  adding: 516/trip_69_part_1.csv (deflated 80%)
  adding: 516/trip_106_part_2.csv (deflated 94%)
  adding: 516/trip_17_part_1.csv (deflated 68%)
  adding: 516/trip_74_part_1.csv (deflated 79%)
  adding: 516/trip_18_part_1.csv (deflated 40%)
  adding: 516/trip_80_part_1.csv (deflated 79%)
  adding: 516/trip_112_part_2.csv (deflated 80%)
  adding: 516/trip_27_part_1.csv (deflated 78%)
  adding: 516/trip_70_part_1.csv (deflated 80%)
  adding: 516/trip_73_part_1.csv (deflated 79%)
  adding: 51

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd



In [3]:
# Step 1: Load the Excel file into a DataFrame
dff = pd.read_excel('/content/drive/MyDrive/Car_Dena_TC+5_26 s 513 ir 10_Data_11_11_2024, 08_20_00_to_11_25_2024, 23_58_00.xlsx')

In [4]:
df1 = dff.copy()

In [5]:
df.columns

NameError: name 'df' is not defined

# **Create seprate file with check time monotonic**



In [ ]:
import os
import pandas as pd

# Sample data loading
# df1 = pd.read_csv("your_data.csv")

# Grouping by 'trip'
grouped = df1.groupby('trip')

# Ensure the folder exists
folder = '515'
if not os.path.exists(folder):
    os.makedirs(folder)

# Function to check if time column is monotonic increasing
def is_monotonic_increasing(time_series):
    return time_series.is_monotonic_increasing

# Iterate through each trip group
for trip, group in grouped:
    # Check if 'time' column is monotonic increasing
    if not is_monotonic_increasing(group['time']):
        # Split the group into two or more, based on where the time decreases or jumps
        # For simplicity, let's split when the time stops increasing
        split_groups = []
        current_group = []
        last_time = None

        for index, row in group.iterrows():
            if last_time is None or row['time'] >= last_time:
                current_group.append(row)
            else:
                # When time decreases or jumps, store the current group and start a new one
                split_groups.append(pd.DataFrame(current_group))
                current_group = [row]
            last_time = row['time']

        # Add the last group
        split_groups.append(pd.DataFrame(current_group))

        # Save each split group as a new CSV file
        for i, split_group in enumerate(split_groups):
            filename = f"{folder}/trip_{trip}_part_{i+1}.csv"
            split_group.to_csv(filename, index=False)

    else:
        # If the time is monotonic, save the group as a CSV file
        filename = f"{folder}/trip_{trip}.csv"
        group.to_csv(filename, index=False)


# **Create seprate file with check time and fuel consumption monotonic**


In [ ]:
import pandas as pd
import os

# Load the Excel file
excel_file = "path/to/your/excel_file.xlsx"  # Replace with the path to your Excel file
output_folder = "path/to/output/folder"  # Replace with the path where you want to save CSV files

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file)

# List of specific trip IDs to filter
specific_trips = [46, 47, 48, 49, 50, 112, 115]

# Filter the DataFrame for the specific trips
filtered_df = df[df['trip'].isin(specific_trips)]

# Group by the "trip" column (replace 'trip' with the actual column name for grouping)
for trip_id, trip_data in filtered_df.groupby('trip'):
    # Create a filename for the trip
    csv_file = os.path.join(output_folder, f"trip_{trip_id}.csv")

    # Save the trip data to a CSV file
    trip_data.to_csv(csv_file, index=False)

    print(f"Saved trip {trip_id} to {csv_file}")


In [6]:
import os
import pandas as pd

# Sample data loading
# df1 = pd.read_csv("your_data.csv")

# Grouping by 'trip'
grouped = df1.groupby('trip')

# Ensure the folder exists
folder = '513'
if not os.path.exists(folder):
    os.makedirs(folder)

# Function to check if both time and fuel consumption are monotonic increasing
def is_monotonic_increasing_with_stability(time_series, fuel_series):
    # Time should be strictly increasing, but fuel consumption can stay constant or increase
    return (time_series.is_monotonic_increasing) and all(fuel_series.diff().ge(0) | fuel_series.diff().eq(0))

# Iterate through each trip group
for trip, group in grouped:
    # Check if both 'time' and 'Trip_fuel_consumption' are monotonic increasing
    if trip==46 or trip ==47 or trip==48 or trip ==49 or trip==50 or trip ==112 or trip==115:
      continue
    else :
      if not is_monotonic_increasing_with_stability(group['time'], group['Trip_fuel_consumption']):
          # Split the group into two or more, based on where time or fuel consumption decreases or jumps
          split_groups = []
          current_group = []
          last_time = None
          last_fuel = None

          for index, row in group.iterrows():
              # Ensure time is strictly increasing and fuel consumption doesn't decrease
              if last_time is None or (row['time'] >= last_time and (row['Trip_fuel_consumption'] >= last_fuel or row['Trip_fuel_consumption'] == last_fuel)):
                  current_group.append(row)
              else:
                  # When either time or fuel consumption decreases or jumps, store the current group and start a new one
                  split_groups.append(pd.DataFrame(current_group))
                  current_group = [row]
              last_time = row['time']
              last_fuel = row['Trip_fuel_consumption']

          # Add the last group
          split_groups.append(pd.DataFrame(current_group))

          # Save each split group as a new CSV file
          for i, split_group in enumerate(split_groups):
              filename = f"{folder}/trip_{trip}_part_{i+1}.csv"
              split_group.to_csv(filename, index=False)

      else:
          # If both time and fuel consumption are monotonic, save the group as a CSV file
          filename = f"{folder}/trip_{trip}.csv"
          group.to_csv(filename, index=False)


In [ ]:
import os
import pandas as pd

# Specify the folder containing the CSV files
folder_path = r"513_600row"

# Loop through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):  # Check if the file is a CSV
        file_path = os.path.join(folder_path, file_name)
        try:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)

            # Check the number of rows in the DataFrame
            if len(df) < 100:
                print(f"Deleting {file_name} (rows: {len(df)})")
                os.remove(file_path)  # Delete the file
            else:
                # print(f"Keeping {file_name} (rows: {len(df)})")
                bb=1
        except Exception as e:
            print(f"Error processing {file_name}: {e}")


In [12]:
import os
import pandas as pd

# Specify the folder containing the CSV files
folder_path = r"513_600row"

# Loop through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):  # Check if the file is a CSV
        file_path = os.path.join(folder_path, file_name)
        try:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)

            # Check the number of rows in the DataFrame
            if len(df) < 100:
                print(f"Deleting {file_name} (rows: {len(df)})")
                os.remove(file_path)  # Delete the file
            else:
                # print(f"Keeping {file_name} (rows: {len(df)})")
                bb=1
        except Exception as e:
            print(f"Error processing {file_name}: {e}")


Deleting trip_96_part_1_2.csv (rows: 51)
Deleting trip_56_part_1_2.csv (rows: 34)
Deleting trip_10_part_1_2.csv (rows: 9)
Deleting trip_39_part_1_5.csv (rows: 7)
Deleting trip_20_part_1_2.csv (rows: 87)
Deleting trip_11_part_1_2.csv (rows: 99)
Deleting trip_113_part_1_10.csv (rows: 60)
Deleting trip_13_part_1_2.csv (rows: 45)
Deleting trip_100_part_1_3.csv (rows: 24)
Deleting trip_38_part_2_6.csv (rows: 34)
Deleting trip_59_part_1_3.csv (rows: 50)
Deleting trip_19_part_1_2.csv (rows: 97)
Deleting trip_24_part_1_2.csv (rows: 79)


# **create in one file**

In [37]:
import os
import pandas as pd

# Folder containing the CSV files
folder = '513'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder) if f.endswith('.csv')]

# Initialize an empty list to store dataframes
dfs = []

# Iterate through each CSV file and read it
for file in csv_files:
    file_path = os.path.join(folder, file)
    df = pd.read_csv(file_path)
    df['Acceleration2'] = df['Vehicle_Speed'].diff().shift(-1).fillna(0)

    dfs.append(df)

# Combine all dataframes into one
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined dataframe to a new CSV file

combined_df.to_csv('combined_trip_data.csv', index=False)

print(f"Combined file created: {combined_file}")


NameError: name 'combined_file' is not defined

# **Delete the rate for create rate**

In [48]:
import os
import pandas as pd

# Define the folder containing the CSV files
folder_path = "513"  # Replace with the correct folder path

# Specify the column to check for null values
column_name = 'Average_fuel_consumption_rate'

# Loop through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        file_path = os.path.join(folder_path, file_name)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Check if the specified column contains any null values
        if df[column_name].isnull().any():
            # If the column has null values, delete the file
            os.remove(file_path)
            print(f"Deleted {file_name} because it contains null values in the '{column_name}' column.")


Deleted trip_3_part_1.csv because it contains null values in the 'Average_fuel_consumption_rate' column.
Deleted trip_52_part_1.csv because it contains null values in the 'Average_fuel_consumption_rate' column.
Deleted trip_6_part_1.csv because it contains null values in the 'Average_fuel_consumption_rate' column.
Deleted trip_51_part_1.csv because it contains null values in the 'Average_fuel_consumption_rate' column.
Deleted trip_7_part_1.csv because it contains null values in the 'Average_fuel_consumption_rate' column.
Deleted trip_5_part_1.csv because it contains null values in the 'Average_fuel_consumption_rate' column.
Deleted trip_53_part_3.csv because it contains null values in the 'Average_fuel_consumption_rate' column.
Deleted trip_4_part_1.csv because it contains null values in the 'Average_fuel_consumption_rate' column.
Deleted trip_2_part_5.csv because it contains null values in the 'Average_fuel_consumption_rate' column.
Deleted trip_8_part_1.csv because it contains null v

# **Save box plot for all colomn**

In [ ]:
import matplotlib.pyplot as plt
import os

output_folder = 'box_plots'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)  # Create folder if it doesn't exist

# Step 3: Create and save a box plot for each numeric column
for column in combined_df.select_dtypes(include='number').columns:
    plt.figure(figsize=(10, 6))  # Optional: adjust figure size
    combined_df.boxplot(column=column)  # Create a box plot for the current column
    plt.title(f'Box Plot for {column}')  # Title for the plot
    plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability
    # Save the box plot to the specified folder
    boxplot_filename = os.path.join(output_folder, f'boxplot_{column}.png')
    plt.savefig(boxplot_filename)  # Save the box plot as a .png file
    plt.close()  # Close the plot

    print(f'Box plot saved as: {boxplot_filename}')

Box plot saved as: box_plots/boxplot_time.png
Box plot saved as: box_plots/boxplot_trip.png
Box plot saved as: box_plots/boxplot_signal.png
Box plot saved as: box_plots/boxplot_Battery_voltage.png
Box plot saved as: box_plots/boxplot_Fuel_level.png
Box plot saved as: box_plots/boxplot_Intake_manifold_absolute_pressure.png
Box plot saved as: box_plots/boxplot_Throttle_position.png
Box plot saved as: box_plots/boxplot_Accelerator_pedal_position.png
Box plot saved as: box_plots/boxplot_Coolant_temperature.png
Box plot saved as: box_plots/boxplot_Vehicle_Speed.png
Box plot saved as: box_plots/boxplot_Engine_speed.png
Box plot saved as: box_plots/boxplot_Target_air_fuel_ratio.png
Box plot saved as: box_plots/boxplot_Current_gear_shift_position_(Current_gear).png
Box plot saved as: box_plots/boxplot_Cumulative_mileage.png
Box plot saved as: box_plots/boxplot_Clutch_torque.png
Box plot saved as: box_plots/boxplot_Trip_fuel_consumption.png
Box plot saved as: box_plots/boxplot_RON_factor.png
Bo

# **Violin plot**

In [38]:
import matplotlib.pyplot as plt
import seaborn as sns  # Import seaborn for violin plots
import os

output_folder = 'violin_plots'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)  # Create folder if it doesn't exist

# Step 3: Create and save a violin plot for each numeric column
for column in combined_df.select_dtypes(include='number').columns:
    plt.figure(figsize=(10, 6))  # Optional: adjust figure size
    sns.violinplot(y=combined_df[column])  # Create a violin plot for the current column
    plt.title(f'Violin Plot for {column}')  # Title for the plot
    plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability (if needed)

    # Save the violin plot to the specified folder
    violinplot_filename = os.path.join(output_folder, f'violinplot_{column}.png')
    plt.savefig(violinplot_filename)  # Save the violin plot as a .png file
    plt.close()  # Close the plot

    print(f'Violin plot saved as: {violinplot_filename}')


Violin plot saved as: violin_plots/violinplot_time.png
Violin plot saved as: violin_plots/violinplot_trip.png
Violin plot saved as: violin_plots/violinplot_signal.png
Violin plot saved as: violin_plots/violinplot_Battery_voltage.png
Violin plot saved as: violin_plots/violinplot_Fuel_level.png
Violin plot saved as: violin_plots/violinplot_Intake_manifold_absolute_pressure.png
Violin plot saved as: violin_plots/violinplot_Throttle_position.png
Violin plot saved as: violin_plots/violinplot_Accelerator_pedal_position.png
Violin plot saved as: violin_plots/violinplot_Coolant_temperature.png
Violin plot saved as: violin_plots/violinplot_Vehicle_Speed.png
Violin plot saved as: violin_plots/violinplot_Engine_speed.png
Violin plot saved as: violin_plots/violinplot_Target_air_fuel_ratio.png
Violin plot saved as: violin_plots/violinplot_Current_gear_shift_position_(Current_gear).png
Violin plot saved as: violin_plots/violinplot_Cumulative_mileage.png
Violin plot saved as: violin_plots/violinplot_

# **Create seprate file with check time and fuel consumption monotonic**

> Add blockquote

> Add blockquote







In [ ]:
!zip -r qani.zip qani

  adding: qani/ (stored 0%)
  adding: qani/qani_107_valid.csv (deflated 73%)
  adding: qani/qani_74_valid.csv (deflated 80%)
  adding: qani/qani_96_valid.csv (deflated 79%)
  adding: qani/qani_186_valid.csv (deflated 82%)
  adding: qani/qani_141_valid.csv (deflated 80%)
  adding: qani/qani_26_valid.csv (deflated 80%)
  adding: qani/qani_148_valid.csv (deflated 79%)
  adding: qani/qani_172_valid.csv (deflated 79%)
  adding: qani/qani_43_valid.csv (deflated 85%)
  adding: qani/qani_31_valid.csv (deflated 87%)
  adding: qani/qani_52_valid.csv (deflated 80%)
  adding: qani/qani_194_valid.csv (deflated 89%)
  adding: qani/qani_120_valid.csv (deflated 79%)
  adding: qani/qani_159_partial.csv (deflated 78%)
  adding: qani/qani_124_valid.csv (deflated 80%)
  adding: qani/qani_131_valid.csv (deflated 79%)
  adding: qani/qani_166_valid.csv (deflated 81%)
  adding: qani/qani_29_valid.csv (deflated 36%)
  adding: qani/qani_198_valid.csv (deflated 83%)
  adding: qani/qani_93_valid.csv (deflated 80%

In [ ]:
!zip -r qani_2.zip qani_2

In [ ]:
!zip -r box_plots.zip box_plots

  adding: box_plots/ (stored 0%)
  adding: box_plots/boxplot_0409.png (deflated 31%)
  adding: box_plots/boxplot_Trip_fuel_consumption.png (deflated 25%)
  adding: box_plots/boxplot_0314.png (deflated 32%)
  adding: box_plots/boxplot_0313.png (deflated 31%)
  adding: box_plots/boxplot_time.png (deflated 27%)
  adding: box_plots/boxplot_Battery_voltage.png (deflated 26%)
  adding: box_plots/boxplot_Engine_speed.png (deflated 26%)
  adding: box_plots/boxplot_Cumulative_mileage.png (deflated 27%)
  adding: box_plots/boxplot_longitude.png (deflated 27%)
  adding: box_plots/boxplot_Target_air_fuel_ratio.png (deflated 24%)
  adding: box_plots/boxplot_Accelerator_pedal_position.png (deflated 25%)
  adding: box_plots/boxplot_signal.png (deflated 27%)
  adding: box_plots/boxplot_Fuel_level.png (deflated 26%)
  adding: box_plots/boxplot_Current_gear_shift_position_(Current_gear).png (deflated 24%)
  adding: box_plots/boxplot_0345.png (deflated 32%)
  adding: box_plots/boxplot_latitude.png (defla

In [ ]:
!zip -r violin_plots.zip violin_plots

  adding: violin_plots/ (stored 0%)
  adding: violin_plots/violinplot_033d.png (deflated 33%)
  adding: violin_plots/violinplot_0409.png (deflated 33%)
  adding: violin_plots/violinplot_Coolant_temperature.png (deflated 13%)
  adding: violin_plots/violinplot_Trip_fuel_consumption.png (deflated 12%)
  adding: violin_plots/violinplot_Throttle_position.png (deflated 13%)
  adding: violin_plots/violinplot_040a.png (deflated 33%)
  adding: violin_plots/violinplot_Battery_voltage.png (deflated 19%)
  adding: violin_plots/violinplot_signal.png (deflated 10%)
  adding: violin_plots/violinplot_Intake_manifold_absolute_pressure.png (deflated 11%)
  adding: violin_plots/violinplot_Cumulative_mileage.png (deflated 14%)
  adding: violin_plots/violinplot_Fuel_level.png (deflated 9%)
  adding: violin_plots/violinplot_0314.png (deflated 33%)
  adding: violin_plots/violinplot_Accelerator_pedal_position.png (deflated 14%)
  adding: violin_plots/violinplot_0313.png (deflated 34%)
  adding: violin_plots/v

In [ ]:
!zip -r valid_trips_csv.zip valid_trips_csv

  adding: valid_trips_csv/ (stored 0%)
  adding: valid_trips_csv/Trip_141.csv (deflated 82%)
  adding: valid_trips_csv/Trip_184.csv (deflated 83%)
  adding: valid_trips_csv/Trip_166.csv (deflated 83%)
  adding: valid_trips_csv/Trip_121.csv (deflated 88%)
  adding: valid_trips_csv/Trip_97.csv (deflated 82%)
  adding: valid_trips_csv/Trip_155.csv (deflated 82%)
  adding: valid_trips_csv/Trip_125.csv (deflated 82%)
  adding: valid_trips_csv/Trip_217.csv (deflated 82%)
  adding: valid_trips_csv/Trip_21.csv (deflated 82%)
  adding: valid_trips_csv/Trip_120.csv (deflated 82%)
  adding: valid_trips_csv/Trip_180.csv (deflated 82%)
  adding: valid_trips_csv/Trip_79.csv (deflated 82%)
  adding: valid_trips_csv/Trip_178.csv (deflated 89%)
  adding: valid_trips_csv/Trip_27.csv (deflated 82%)
  adding: valid_trips_csv/Trip_112.csv (deflated 82%)
  adding: valid_trips_csv/Trip_63.csv (deflated 85%)
  adding: valid_trips_csv/Trip_115.csv (deflated 83%)
  adding: valid_trips_csv/Trip_58.csv (deflated 

# **Altitude**

In [8]:
import pandas as pd
import requests
import time
import os
from tqdm import tqdm  # Import tqdm for the file-level progress bar

# Path to the folder containing CSV files
folder_path = "513"  # Update to your folder path

# Initialize last received altitude
last_received_altitude = None

# Function to get altitude
def get_altitude(lat, lon, retries=10):
    global last_received_altitude  # Use the last received altitude as fallback
    url = f"https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}"
    url = f'https://www.elevation-api.eu/v1/elevation?pts=[[{lat},{lon}]]'
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                elevation_data = response.json()
                if 'elevations' in elevation_data and elevation_data['elevations']:
                    altitude = elevation_data['elevations'][0]  # Extracting the first elevation value
                    last_received_altitude = altitude  # Update last successful altitude
                    return altitude
            elif response.status_code == 504:
                return last_received_altitude  # Use last received altitude on 504 error
        except requests.exceptions.RequestException:
            time.sleep(2)  # Wait for 2 seconds before retrying
    return last_received_altitude  # Use last received altitude if all retries fail

# Helper function to get altitude for a block of 100 rows
def get_altitude_for_block(df, start_index):
    for i in range(start_index, min(start_index + 100, len(df))):
        lat = df.at[i, 'latitude']
        lon = df.at[i, 'longitude']

        # Skip rows with missing or invalid latitude/longitude
        if pd.isna(lat) or pd.isna(lon) or lat == 0 or lon == 0:
            continue

        altitude = get_altitude(lat, lon)
        if altitude is not None:
            return altitude
    return None

# Process each CSV file in the specified folder
for filename in tqdm(os.listdir(folder_path), desc="Processing files"):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        data_df = pd.read_csv(file_path)

        # Check if 'latitude' and 'longitude' columns exist in the dataframe
        if 'latitude' not in data_df.columns or 'longitude' not in data_df.columns:
            continue  # Skip this file if columns are missing

        # Initialize the 'correct_altitude' column with None values
        data_df['correct_altitude'] = None

        # Process rows in blocks of 100
        for i in range(0, len(data_df), 100):
            altitude = get_altitude_for_block(data_df, i)

            # Adjust the end of the range to ensure it's within bounds and handle small blocks
            block_end = min(i + 99, len(data_df) - 1)

            # If the altitude is None and the block size is 1 (e.g., last row), skip setting altitude
            if altitude is None and (block_end == i):
                continue

            # Set the altitude for the current block
            data_df.loc[i:block_end, 'correct_altitude'] = altitude

        # Save the updated dataframe to a new CSV file, retaining the same name
        output_file_path = os.path.join(folder_path, filename)  # Save to the same file
        data_df.to_csv(output_file_path, index=False)

print("Updated files saved successfully.")


Processing files: 100%|██████████| 104/104 [07:59<00:00,  4.61s/it]

Updated files saved successfully.


In [10]:
import pandas as pd
import os
from tqdm import tqdm  # Import tqdm for the file-level progress bar
# Path to the folder containing CSV files
folder_path = "513"

# Process each CSV file in the specified folder
for filename in tqdm(os.listdir(folder_path), desc="Processing files"):
    if filename.endswith('.csv'):
        print(filename)
        file_path = os.path.join(folder_path, filename)
        print(f"Processing file: {filename}")

        # Read the CSV file
        data_df = pd.read_csv(file_path)
        data_df = data_df.sort_values(by='time')
        data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')

        # Sort the DataFrame by 'time' and 'trip'


        # Initialize the 'slope' column with 0.0
        data_df['slope'] = 0.0

        # Group by 'trip' and calculate slope for each group
        for trip, group in data_df.groupby('trip'):
            # Check if the group has at least 100 rows
            num_rows = len(group)
            for i in range(0, num_rows - 100, 100):
                start_altitude = group.iloc[i]['correct_altitude']
                end_altitude = group.iloc[i + 100]['correct_altitude']
                start_mileage = group.iloc[i]['Cumulative_mileage']
                end_mileage = group.iloc[i + 100]['Cumulative_mileage']

                if end_mileage - start_mileage != 0:
                    slope = (end_altitude - start_altitude) / (end_mileage - start_mileage) / 10
                    data_df.loc[group.index[i:i + 99], 'slope'] = slope

            # Calculate slope for remaining rows if less than 100 remain
            if num_rows % 100 != 0:
                last_block_start = num_rows - (num_rows % 100)
                if last_block_start > 0:
                    start_altitude = group.iloc[last_block_start - 1]['correct_altitude']
                    end_altitude = group.iloc[num_rows - 1]['correct_altitude']
                    start_mileage = group.iloc[last_block_start - 1]['Cumulative_mileage']
                    end_mileage = group.iloc[num_rows - 1]['Cumulative_mileage']

                    if end_mileage - start_mileage != 0:
                        slope = (end_altitude - start_altitude) / (end_mileage - start_mileage) / 10
                        data_df.loc[group.index[last_block_start - 1:num_rows - 1], 'slope'] = slope

        # Define output file path
        # output_file_path = os.path.join(folder_path, f"{filename[:-4]}_slope_added.csv")
        output_file_path = os.path.join(folder_path, filename)

        # Save the updated DataFrame to a new CSV file
        data_df.to_csv(output_file_path, index=False)

print("All files processed successfully.")


Processing files:   0%|          | 0/104 [00:00<?, ?it/s]<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')
<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')
<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')


trip_62_part_1.csv
Processing file: trip_62_part_1.csv
trip_90_part_2.csv
Processing file: trip_90_part_2.csv
trip_113_part_2.csv
Processing file: trip_113_part_2.csv


Processing files:   3%|▎         | 3/104 [00:00<00:10, 10.05it/s]<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')
<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')
<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')
<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df[

trip_37_part_1.csv
Processing file: trip_37_part_1.csv
trip_83_part_2.csv
Processing file: trip_83_part_2.csv
trip_69_part_1.csv
Processing file: trip_69_part_1.csv
trip_106_part_2.csv
Processing file: trip_106_part_2.csv
trip_74_part_1.csv
Processing file: trip_74_part_1.csv
trip_80_part_1.csv
Processing file: trip_80_part_1.csv
trip_27_part_1.csv
Processing file: trip_27_part_1.csv
trip_70_part_1.csv
Processing file: trip_70_part_1.csv


<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')
Processing files:  12%|█▏        | 12/104 [00:00<00:03, 23.84it/s]<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')
<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')
<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df

...

trip_28_part_1.csv
Processing file: trip_28_part_1.csv
trip_45_part_1.csv
Processing file: trip_45_part_1.csv


<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')
Processing files:  91%|█████████▏| 95/104 [00:04<00:00, 14.56it/s]<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')
<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')
<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df

trip_21_part_1.csv
Processing file: trip_21_part_1.csv
trip_2_part_5.csv
Processing file: trip_2_part_5.csv
trip_75_part_1.csv
Processing file: trip_75_part_1.csv
trip_87_part_1.csv
Processing file: trip_87_part_1.csv
trip_107_part_1.csv
Processing file: trip_107_part_1.csv
trip_68_part_1.csv
Processing file: trip_68_part_1.csv


<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')
<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')
<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='bfill')
<ipython-input-10-fd92ed7521a1>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df['correct_altitude'] = data_df['correct_altitude'].fillna(method='

trip_101_part_1.csv
Processing file: trip_101_part_1.csv
trip_8_part_1.csv
Processing file: trip_8_part_1.csv
trip_63_part_1.csv
Processing file: trip_63_part_1.csv
trip_91_part_1.csv
Processing file: trip_91_part_1.csv
All files processed successfully.


In [ ]:
import os
import pandas as pd

# Folder containing the CSV files
folder = '516'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder) if f.endswith('.csv')]

# Initialize an empty list to store dataframes
dfs = []

# Iterate through each CSV file and read it
for file in csv_files:
    file_path = os.path.join(folder, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# Combine all dataframes into one
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined dataframe to a new CSV file

combined_df.to_csv('combined_trip_data_alt.csv', index=False)

print(f"Combined file created: {combined_file}")


Combined file created: 516/combined_trip_data.csv


# **Valid trip**

# **Devide to 600Rows**

In [ ]:
!unzip -qq '/content/drive/MyDrive/valid_trips_csv_cor_slop.zip'

In [11]:
import pandas as pd
import os

def split_csv_file(input_file, output_file_prefix, rows_per_file):
    df = pd.read_csv(input_file)

    total_rows = len(df)
    num_files = (total_rows // rows_per_file) + (1 if total_rows % rows_per_file != 0 else 0)

    for i in range(num_files):
        start_row = i * rows_per_file
        end_row = start_row + rows_per_file

        # Create a subset DataFrame
        subset_df = df.iloc[start_row:end_row]

        # Generate output file name
        output_file = f"{output_file_prefix}_{i + 1}.csv"

        # Save the subset DataFrame to a new CSV file
        subset_df.to_csv(output_file, index=False)

    print(f"{input_file} split into {num_files} files, including the last segment if under {rows_per_file} rows.")

# Path to the folder containing CSV files
folder_path = '513'
folder_path1 = '513_600row'
# Process each CSV file in the specified folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        input_file_path = os.path.join(folder_path, filename)
        output_file_prefix = os.path.join(folder_path1, filename[:-4])  # Remove .csv for output file naming
        rows_per_file = 600

        split_csv_file(input_file_path, output_file_prefix, rows_per_file)

print("All files processed successfully.")


513/trip_62_part_1.csv split into 1 files, including the last segment if under 600 rows.
513/trip_90_part_2.csv split into 1 files, including the last segment if under 600 rows.
513/trip_113_part_2.csv split into 12 files, including the last segment if under 600 rows.
513/trip_37_part_1.csv split into 1 files, including the last segment if under 600 rows.
513/trip_83_part_2.csv split into 1 files, including the last segment if under 600 rows.
513/trip_69_part_1.csv split into 2 files, including the last segment if under 600 rows.
513/trip_106_part_2.csv split into 1 files, including the last segment if under 600 rows.
513/trip_74_part_1.csv split into 1 files, including the last segment if under 600 rows.
513/trip_80_part_1.csv split into 1 files, including the last segment if under 600 rows.
513/trip_27_part_1.csv split into 1 files, including the last segment if under 600 rows.
513/trip_70_part_1.csv split into 3 files, including the last segment if under 600 rows.
513/trip_73_part_1

In [ ]:
def split_csv_file(input_file):
    df = pd.read_csv(input_file)

    print(input_file,len(df))


folder_path1 = '/content/valid_trips_csv_600row'
# Process each CSV file in the specified folder
for filename in os.listdir(folder_path1):
    if filename.endswith('.csv'):
        input_file_path = os.path.join(folder_path1, filename)

        split_csv_file(input_file_path)

print("All files processed successfully.")

In [ ]:
!zip -r valid_trips_csv_600row.zip valid_trips_csv_600row

  adding: valid_trips_csv_600row/ (stored 0%)
  adding: valid_trips_csv_600row/Trip_35_1.csv (deflated 80%)
  adding: valid_trips_csv_600row/Trip_130_4.csv (deflated 86%)
  adding: valid_trips_csv_600row/Trip_90_18.csv (deflated 81%)
  adding: valid_trips_csv_600row/Trip_214_2.csv (deflated 81%)
  adding: valid_trips_csv_600row/Trip_134_5.csv (deflated 92%)
  adding: valid_trips_csv_600row/Trip_111_5.csv (deflated 82%)
  adding: valid_trips_csv_600row/Trip_185_2.csv (deflated 82%)
  adding: valid_trips_csv_600row/Trip_35_4.csv (deflated 81%)
  adding: valid_trips_csv_600row/Trip_134_2.csv (deflated 80%)
  adding: valid_trips_csv_600row/Trip_172_2.csv (deflated 81%)
  adding: valid_trips_csv_600row/Trip_144_2.csv (deflated 80%)
  adding: valid_trips_csv_600row/Trip_68_1.csv (deflated 81%)
  adding: valid_trips_csv_600row/Trip_102_3.csv (deflated 80%)
  adding: valid_trips_csv_600row/Trip_96_1.csv (deflated 80%)
  adding: valid_trips_csv_600row/Trip_180_4.csv (deflated 81%)
  adding: val

# **Augment V7**

In [ ]:
import os

def count_csv_files(folder_path):
    # List all files in the specified folder
    files = os.listdir(folder_path)

    # Filter for CSV files
    csv_files = [file for file in files if file.endswith('.csv')]

    # Count the number of CSV files
    count = len(csv_files)

    print(f"Number of CSV files in '{folder_path}': {count}")
    return count

# Path to the folder containing CSV files
folder_path = '/content/data_aug(3_slices_with_repeated)_cluster_5'

# Call the function to count CSV files
count_csv_files(folder_path)

Number of CSV files in '/content/data_aug(3_slices_with_repeated)_cluster_5': 50127


50127

In [ ]:
import pandas as pd
import os
import itertools

def adjust_sequence(prev_value, slice_values):
    adjusted_values = [prev_value]
    for i in range(len(slice_values)):
        delta = slice_values[i] - slice_values[i - 1] if i > 0 else 0
        adjusted_values.append(adjusted_values[-1] + delta)
    return adjusted_values[1:]

# def process_combination(slices):
#     processed_data = pd.DataFrame()
#     num_slices = len(slices)
#     for i in range(num_slices):
#         if i > 0:
#             for column in ['Time', 'Cumulative mileage', 'Trip fuel consumption']:
#                 prev_value = processed_data[column].iloc[-1]
#                 slices[i][column] = adjust_sequence(prev_value, list(slices[i][column]))
#             slices[i] = slices[i].iloc[1:]
#         processed_data = pd.concat([processed_data, slices[i]])
#     return processed_data


def process_combination(slices):
    processed_data = pd.DataFrame()
    num_slices = len(slices)
    for i in range(num_slices):
        if i > 0:
            for column in ['Time', 'Cumulative mileage', 'Trip fuel consumption']:
                try:
                    prev_value = processed_data[column].iloc[-1]
                except IndexError:
                    # Handle empty processed_data: You could either skip this part or use a default value
                    prev_value = slices[i][column].iloc[0]  # Using first value of current slice as fallback
                    print(
                        f"Warning: processed_data for column '{column}' is empty. Using first value from current slice.")

                slices[i][column] = adjust_sequence(prev_value, list(slices[i][column]))
            slices[i] = slices[i].iloc[1:]  # Skip first row of the current slice (as it is already adjusted)
        processed_data = pd.concat([processed_data, slices[i]])
    return processed_data

num_slices = 2


source_folder = '/content/valid_trips_csv_600row'
target_folder = 'data_aug(3_slices_with_repeated)_cluster_5'

os.makedirs(target_folder, exist_ok=True)

driver_files = {}
for filename in os.listdir(source_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(source_folder, filename)
        df = pd.read_csv(file_path)
        # comment this for offline data loggers
        df.rename(columns={'Engine_speed': 'Engine speed',
                           'time': 'Time',
                           'Vehicle_Speed': 'Speed',
                           'Trip_fuel_consumption': 'Trip fuel consumption',
                           'Throttle_position': 'Throttle position',
                           'Accelerator_pedal_position': 'Accelerator pedal position',
                           'Cumulative_mileage': 'Cumulative mileage'
                           }, inplace=True)

        min_mileage = df['Cumulative mileage'].min()
        max_mileage = df['Cumulative mileage'].max()
        slice_size = (max_mileage - min_mileage) / num_slices

        # driver_name = driver_data.loc[driver_data['fileName'].str.contains(filename[:-4], na=False), 'driver_name_english'].values[0]
        driver_name = filename[:-4]
        mileage_slices = [df[(df['Cumulative mileage'] >= min_mileage + i * slice_size) &
                             (df['Cumulative mileage'] < min_mileage + (i + 1) * slice_size)]
                          for i in range(num_slices)]

        mileage_slices[-1] = pd.concat([mileage_slices[-1], df[df['Cumulative mileage'] == max_mileage]])
        driver_files[driver_name] = mileage_slices

driver_combinations = list(itertools.product(driver_files.keys(), repeat=num_slices))

file_count = 0
# for combination in driver_combinations:
#     slices = [driver_files[driver][slice_index] for driver, slice_index in zip(combination, range(num_slices))]
#     combined_data = process_combination(slices)
#     output_file = os.path.join(target_folder,
#                                f"{'_'.join([f'{driver}(slice_{slice_index + 1})' for driver, slice_index in zip(combination, range(num_slices))])}.csv")
#     combined_data.to_csv(output_file, index=False)
#     file_count += 1

for combination in driver_combinations:
    slices = [driver_files[driver][slice_index] for driver, slice_index in zip(combination, range(num_slices))]
    combined_data = process_combination(slices)

    # Only save if the length of the combined DataFrame is 600 or more
    if len(combined_data) >= 600:
        output_file = os.path.join(target_folder,
                                   f"{'_'.join([f'{driver}(slice_{slice_index + 1})' for driver, slice_index in zip(combination, range(num_slices))])}.csv")
        combined_data.to_csv(output_file, index=False)
        file_count += 1

Streaming output truncated to the last 5000 lines.


In [ ]:
!zip -r 'data_aug(3_slices_with_repeated)_qani.zip' 'data_aug(3_slices_with_repeated)_cluster_5'

Streaming output truncated to the last 5000 lines.
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_164_1(slice_1)_Trip_141_5(slice_2).csv (deflated 83%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_161_2(slice_1)_Trip_90_27(slice_2).csv (deflated 82%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_184_2(slice_1)_Trip_14_6(slice_2).csv (deflated 81%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_186_4(slice_1)_Trip_137_3(slice_2).csv (deflated 85%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_217_1(slice_1)_Trip_90_36(slice_2).csv (deflated 83%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_103_2(slice_1)_Trip_96_3(slice_2).csv (deflated 82%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_21_1(slice_1)_Trip_57_24(slice_2).csv (deflated 81%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_79_3(slice_1)_Trip_178_1(slice_2).csv (deflated 85%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from multiprocessing import Pool
from sklearn.preprocessing import MinMaxScaler
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import joblib

SEQUENCE_LENGTH = 600
BATCH_SIZE = 64
EPOCHS = 250
LEARNING_RATE = 1e-5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data processing function
def process_file(file_path):
    df = pd.read_csv(file_path)
    if df.shape[0] >= 600:
        df['Time'] = df['Time'] - df['Time'].iloc[0]
        df['Momentary fuel consumption'] = df['Trip fuel consumption'].diff().fillna(0)
        df['Acceleration'] = df['Speed'].diff().fillna(0)
        df['Adjusted_gear_position'] = df['Current_gear_shift_position_(Current_gear)'].replace({13: 0.2, 14: 1})

        features = df[['Adjusted_gear_position', 'Speed', 'slope', 'Acceleration','Coolant_temperature']]
        target = df['Momentary fuel consumption']
        return features.values, target.values
    else:
        return None, None

# Function to slice data into chunks and append to lists
def slice_and_append(features, target, sequence_length):
    num_chunks = len(features) // sequence_length
    X_list, y_list = [], []

    for i in range(num_chunks):
        start_idx = i * sequence_length
        end_idx = start_idx + sequence_length
        sliced_features = features[start_idx:end_idx]
        sliced_target = target[start_idx:end_idx]
        X_list.append(sliced_features)
        y_list.append(sliced_target)

    return X_list, y_list

# Helper function to process multiple files
def process_and_slice(file_path):
    features, target = process_file(file_path)
    if features is not None and target is not None:
        return slice_and_append(features, target, SEQUENCE_LENGTH)
    return [], []

# Function to process a folder in parallel
def process_folder_parallel(folder_path, num_workers=4):
    all_X, all_y = [], []
    file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

    with Pool(num_workers) as p:
        results = p.map(process_and_slice, file_paths)

    # Unpacking the processed slices
    for X_list, y_list in results:
        all_X.extend(X_list)
        all_y.extend(y_list)

    return all_X, all_y
X_original = []
y_original = []
# Main processing logic
def main(base_folder_path):

    X_augmented = []
    y_augmented = []

    for i in range(7):
        if i == 5 :
            folder_path = os.path.join(base_folder_path, f'data_aug(3_slices_with_repeated)_cluster_{i}')
            X_aug, y_aug = process_folder_parallel(folder_path, num_workers=4)
            X_augmented.extend(X_aug)
            y_augmented.extend(y_aug)

    return X_augmented, y_augmented

# Example call to main
base_folder_path = '/content/'
X_augmented, y_augmented = main(base_folder_path)


In [ ]:

# Function to pad or truncate sequences
def pad_or_truncate(sequence, length):
    if len(sequence) > length:
        return sequence[:length]
    elif len(sequence) < length:
        return np.pad(sequence, ((0, length - len(sequence)), (0, 0)), mode='constant')
    else:
        return sequence



In [ ]:
X_original = []
y_original = []
# Apply padding/truncating to ensure consistent sequence length
X_original = [pad_or_truncate(x, SEQUENCE_LENGTH) for x in X_original]
y_original = [pad_or_truncate(y.reshape(-1, 1), SEQUENCE_LENGTH) for y in y_original]
X_augmented = [pad_or_truncate(x, SEQUENCE_LENGTH) for x in X_augmented]
y_augmented = [pad_or_truncate(y.reshape(-1, 1), SEQUENCE_LENGTH) for y in y_augmented]


In [ ]:
np.array(X_augmented).shape

(50127, 600, 5)

In [ ]:
import numpy as np

# Convert lists to numpy arrays if they aren't already
X_original = np.array(X_original)
y_original = np.array(y_original)
X_augmented = np.array(X_augmented)
y_augmented = np.array(y_augmented)

# Saving to npz file
np.savez(f'data{SEQUENCE_LENGTH}_coolant_qani.npz', X_original=X_original, y_original=y_original, X_augmented=X_augmented, y_augmented=y_augmented)


# **Augment time and coolant**



```
# This is formatted as code
```

# **seperate validations**

In [ ]:
import os
import shutil
import random
import glob

source = '/content/qani/'
destin = '/content/valid_qani'
# os.makedirs(destin)


csv_files = glob.glob(os.path.join(source ,'*.csv'))

num_file_move = 10
files_to_move = random.sample(csv_files ,num_file_move )

for file_name in files_to_move :
  sours_file = os.path.join(source , file_name)
  destinfile = os.path.join(destin ,file_name)

  shutil.move(sours_file ,destinfile )
  print(f'{file_name} okkkk')

/content/qani/qani_9_partial_1.csv okkkk
/content/qani/qani_32_valid.csv okkkk
/content/qani/qani_54_partial_1.csv okkkk
/content/qani/qani_64_partial_last.csv okkkk
/content/qani/qani_92_partial_last.csv okkkk
/content/qani/qani_48_valid.csv okkkk
/content/qani/qani_47_valid.csv okkkk
/content/qani/qani_130_valid.csv okkkk
/content/qani/qani_27_valid.csv okkkk
/content/qani/qani_142_valid.csv okkkk


# **aug**

In [ ]:
import os
import pandas as pd
import itertools
import random  # For random sampling
from tqdm import tqdm  # For progress tracking

def adjust_sequence(prev_value, slice_values):
    adjusted_values = [prev_value]
    for i in range(len(slice_values)):
        delta = slice_values[i] - slice_values[i - 1] if i > 0 else 0
        adjusted_values.append(adjusted_values[-1] + delta)
    return adjusted_values[1:]

def process_combination(slices):
    processed_data = pd.DataFrame()
    num_slices = len(slices)

    for i in range(num_slices):
        if i > 0:
            # Check if processed_data is not empty before accessing the last row
            if processed_data.empty:
                continue

            # Check Coolant_temperature condition before adjusting sequence
            coolant_temp_prev = processed_data['Coolant_temperature'].iloc[-1]
            coolant_temp_current = slices[i]['Coolant_temperature'].iloc[0]

            # Skip slice if Coolant_temperature difference is greater than 10°C
            if abs(coolant_temp_current - coolant_temp_prev) > 10:
                continue  # Skip this slice if the temperature difference is more than 10°C

            # Adjust sequences for specified columns
            for column in ['Time', 'Cumulative mileage', 'Trip fuel consumption']:
                if not processed_data.empty:
                    prev_value = processed_data[column].iloc[-1]
                else:
                    prev_value = slices[i][column].iloc[0]  # Fallback in case of empty processed_data

                slices[i].loc[:, column] = adjust_sequence(prev_value, list(slices[i][column]))

            slices[i] = slices[i].iloc[1:]  # Skip the first row of the current slice (already adjusted)

        processed_data = pd.concat([processed_data, slices[i]], ignore_index=True)

    return processed_data

# Number of slices for each combination
num_slices = 3

# Define source and target folders
source_folder = '/content/qani'
target_folder = 'data_aug_new_fine_tune_791'
os.makedirs(target_folder, exist_ok=True)

# Process files in the source folder
driver_files = {}
for filename in tqdm(os.listdir(source_folder), desc="Processing files"):
    if filename.endswith('.csv'):
        file_path = os.path.join(source_folder, filename)
        df = pd.read_csv(file_path)

        # Rename columns if necessary
        df.rename(columns={
            'Engine_speed': 'Engine speed',
            'time': 'Time',
            'Vehicle_Speed': 'Speed',
            'Trip_fuel_consumption': 'Trip fuel consumption',
            'Throttle_position': 'Throttle position',
            'Accelerator_pedal_position': 'Accelerator pedal position',
            'Cumulative_mileage': 'Cumulative mileage',
            'Coolant_temp': 'Coolant_temperature'
        }, inplace=True)

        # Calculate slice size
        slice_size = len(df) // num_slices

        driver_name = filename[:-4]  # Extract driver name from filename

        # Slice the dataframe
        mileage_slices = [df.iloc[i * slice_size:(i + 1) * slice_size] for i in range(num_slices)]
        mileage_slices[-1] = pd.concat([mileage_slices[-1], df.iloc[(num_slices * slice_size):]])

        driver_files[driver_name] = mileage_slices

# Generate combinations and process the slices
driver_combinations = list(itertools.product(driver_files.keys(), repeat=num_slices))

file_count = 0
total_rows = 0
target_rows = 10000000000000000000000  # The total number of rows to aim for

# Randomly shuffle the driver combinations to process them in a random order
random.shuffle(driver_combinations)

for combination in tqdm(driver_combinations, desc="Processing combinations"):
    slices = [driver_files[driver][slice_index] for driver, slice_index in zip(combination, range(num_slices))]
    combined_data = process_combination(slices)

    # Only save if the combined DataFrame has enough rows and total_rows is less than 50,000
    if len(combined_data) >= 600 :
        output_file = os.path.join(target_folder, f"{'_'.join([f'{driver}(slice_{slice_index + 1})' for driver, slice_index in zip(combination, range(num_slices))])}.csv")
        combined_data.to_csv(output_file, index=False)
        total_rows += len(combined_data)

    # Break if we have reached the target of 50,000 rows
    if total_rows >= target_rows:
        print(f"Reached target of {target_rows} rows. Stopping further processing.")
        break


print(f"Processed and saved {file_count} files with a total of {total_rows} rows.")


Processing combinations: 100%|██████████| 35937/35937 [11:37<00:00, 51.49it/s]

Processed and saved 0 files with a total of 14411556 rows.


# **new randomize augment**

In [56]:
import os
import pandas as pd
import itertools
import random  # For random sampling
from tqdm import tqdm  # For progress tracking

def adjust_sequence(prev_value, slice_values):
    adjusted_values = [prev_value]
    for i in range(len(slice_values)):
        delta = slice_values[i] - slice_values[i - 1] if i > 0 else 0
        adjusted_values.append(adjusted_values[-1] + delta)
    return adjusted_values[1:]

def process_combination(slices):
    processed_data = pd.DataFrame()
    num_slices = len(slices)

    for i in range(num_slices):
        if i > 0:
            # Check if processed_data is not empty before accessing the last row
            if processed_data.empty:
                continue

            # Check Coolant_temperature condition before adjusting sequence
            coolant_temp_prev = processed_data['Coolant_temperature'].iloc[-1]
            coolant_temp_current = slices[i]['Coolant_temperature'].iloc[0]

            # Skip slice if Coolant_temperature difference is greater than 10°C
            if abs(coolant_temp_current - coolant_temp_prev) > 10:
                continue  # Skip this slice if the temperature difference is more than 10°C

            # Adjust sequences for specified columns
            for column in ['Time', 'Cumulative mileage', 'Trip fuel consumption']:
                if not processed_data.empty:
                    prev_value = processed_data[column].iloc[-1]
                else:
                    prev_value = slices[i][column].iloc[0]  # Fallback in case of empty processed_data

                slices[i].loc[:, column] = adjust_sequence(prev_value, list(slices[i][column]))

            slices[i] = slices[i].iloc[1:]  # Skip the first row of the current slice (already adjusted)

        processed_data = pd.concat([processed_data, slices[i]], ignore_index=True)

    return processed_data

# Number of slices for each combination
num_slices = 3

# Define source and target folders
source_folder = '513'
target_folder = '513_aug_3slices_rate'
os.makedirs(target_folder, exist_ok=True)

# Process files in the source folder
driver_files = {}
for filename in tqdm(os.listdir(source_folder), desc="Processing files"):
    if filename.endswith('.csv'):
        file_path = os.path.join(source_folder, filename)
        df = pd.read_csv(file_path)

        # Rename columns if necessary
        df.rename(columns={
            'Engine_speed': 'Engine speed',
            'time': 'Time',
            'Vehicle_Speed': 'Speed',
            'Trip_fuel_consumption': 'Trip fuel consumption',
            'Throttle_position': 'Throttle position',
            'Accelerator_pedal_position': 'Accelerator pedal position',
            'Cumulative_mileage': 'Cumulative mileage',
            'Coolant_temp': 'Coolant_temperature'
        }, inplace=True)

        # Calculate slice size
        slice_size = len(df) // num_slices

        driver_name = filename[:-4]  # Extract driver name from filename

        # Slice the dataframe
        mileage_slices = [df.iloc[i * slice_size:(i + 1) * slice_size] for i in range(num_slices)]
        mileage_slices[-1] = pd.concat([mileage_slices[-1], df.iloc[(num_slices * slice_size):]])

        driver_files[driver_name] = mileage_slices

# Generate combinations and process the slices
driver_combinations = list(itertools.product(driver_files.keys(), repeat=num_slices))

file_count = 0
total_rows = 0
target_rows = 100000000000000  # The total number of rows to aim for

# Randomly shuffle the driver combinations to process them in a random order
random.shuffle(driver_combinations)
ii = 0
for combination in tqdm(driver_combinations, desc="Processing combinations"):
    slices = [random.choice(driver_files[driver]) for driver in combination]
    combined_data = process_combination(slices)

    # Only save if the combined DataFrame has enough rows and total_rows is less than 50,000
    if len(combined_data) >= 600 :
        output_file = os.path.join(target_folder, f"{'_'.join([f'{driver}(slice_{slice_index + 1})' for driver, slice_index in zip(combination, range(num_slices))])}.csv")
        combined_data.to_csv(output_file, index=False)
        total_rows += len(combined_data)
        ii+=1
    if ii %100==0:
      print(total_rows)
    # Break if we have reached the target of 50,000 rows
    if total_rows >= target_rows:
        print(f"Reached target of {target_rows} rows. Stopping further processing.")
        break

print(f"Processed and saved {file_count} files with a total of {total_rows} rows.")


Processing combinations:   0%|          | 9/830584 [00:00<3:11:00, 72.47it/s]

0
0
0
0
0
0


Processing combinations:   0%|          | 243/830584 [00:07<5:28:49, 42.09it/s]

134157
134157
134157


Processing combinations:   0%|          | 448/830584 [00:13<5:44:22, 40.18it/s]

...

49317013


Processing combinations:   9%|▉         | 76468/830584 [40:02<5:33:52, 37.64it/s]

49454471


Processing combinations:   9%|▉         | 76612/830584 [40:07<6:34:56, 31.82it/s]


KeyboardInterrupt: 

In [ ]:
!zip -r '/content/drive/MyDrive/514_augment_2slices.zip' '514_augment_2slices'

Streaming output truncated to the last 5000 lines.
  adding: 514_augment_2slices/trip_78_part_2(slice_1)_trip_46_part_1(slice_2)_88a4d9ea-51b6-4b43-8f91-8a205cacfaec.csv (deflated 82%)
  adding: 514_augment_2slices/trip_75_part_2(slice_1)_trip_38_part_2(slice_2)_7215c4ca-bd61-4453-b84a-8b747d7346ef.csv (deflated 89%)
  adding: 514_augment_2slices/trip_114_part_2(slice_1)_trip_70_part_1(slice_2)_4d4e6e6a-c23d-42fd-9f8c-11a53201a754.csv (deflated 80%)
  adding: 514_augment_2slices/trip_90_part_2(slice_1)_trip_28_part_1(slice_2)_22723f13-c342-4a6e-b18b-ceca9e07c109.csv (deflated 84%)
  adding: 514_augment_2slices/trip_73_part_1(slice_1)_trip_108_part_1(slice_2)_265ea1d7-74bc-43a9-8a20-2ee78433e311.csv (deflated 79%)
  adding: 514_augment_2slices/trip_57_part_1(slice_1)_trip_38_part_2(slice_2)_1a8387c8-c0ff-4794-aacf-3c254780003b.csv (deflated 87%)
  adding: 514_augment_2slices/trip_24_part_1(slice_1)_trip_7_part_1(slice_2)_636eadd2-3933-402e-98f6-cd9cc2909992.csv (deflated 80%)
  adding: 

In [24]:
import os

# Path to the folder you want to count the files in
folder_path = '/content/513_aug_3slices'

# Count only CSV files in the folder
csv_file_count = len([f for f in os.listdir(folder_path) if f.endswith('.csv') and os.path.isfile(os.path.join(folder_path, f))])

print(f"Number of CSV files in the folder: {csv_file_count}")



Number of CSV files in the folder: 27835


# **create npz file**

In [42]:
import os
import pandas as pd

# Path to the folder containing your CSV files
folder_path = '/content/513_aug_2slices'

# List of columns to check for NaN values
columns_to_check = ['Throttle position','Accelerator pedal position','Coolant_temperature','Speed','Engine speed', 'Current_gear_shift_position_(Current_gear)','correct_altitude',  'slope']

# Iterate over each CSV file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)

        # Load the CSV file
        df = pd.read_csv(file_path)

        # Check which columns have NaN values
        nan_columns = df[columns_to_check].isnull().any()
        columns_with_nan = nan_columns[nan_columns].index.tolist()

        # If any columns contain NaN, print the file name and columns
        if columns_with_nan:
            print(f"NaN values found in file: {filename}")
            print(f"Columns with NaN: {columns_with_nan}")


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [2]:
import os
import numpy as np
import pandas as pd
# import torch
from multiprocessing import Pool
from sklearn.preprocessing import MinMaxScaler
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
import joblib
from tqdm import tqdm  # Import tqdm for progress tracking

SEQUENCE_LENGTH = 600
BATCH_SIZE = 64
EPOCHS = 250
LEARNING_RATE = 1e-5
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data processing function
def process_file(file_path):
    df = pd.read_csv(file_path)
    if not df.empty :
      df.rename(columns={
          'Engine_speed': 'Engine speed',
          'time': 'Time',
          'Vehicle_Speed': 'Speed',
          'Trip_fuel_consumption': 'Trip fuel consumption',
          'Throttle_position': 'Throttle position',
          'Accelerator_pedal_position': 'Accelerator pedal position',
          'Cumulative_mileage': 'Cumulative mileage',
          'Coolant_temp': 'Coolant_temperature'
      }, inplace=True)
      df['Time'] = df['Time'] - df['Time'].iloc[0]
      df['Momentary fuel consumption1'] = df['Trip fuel consumption'].diff().fillna(0)
      df['Momentary fuel consumption2'] = df['Trip fuel consumption'].diff().shift(-1).fillna(0)
      df['Acceleration1'] = df['Speed'].diff().fillna(0)
      df['Acceleration2'] = df['Speed'].diff().shift(-1).fillna(0)
      df['Current_gear_shift_position_(Current_gear)'] = df['Current_gear_shift_position_(Current_gear)'].replace({13: 0, 14: 1})
      # features = df[['Adjusted_gear_position', 'Speed', 'slope', 'Acceleration', 'Coolant_temperature']]


      df = df.iloc[:-5]


      features = df[['Acceleration1','Acceleration2','Throttle position','Accelerator pedal position','Coolant_temperature','Speed','Engine speed', 'Current_gear_shift_position_(Current_gear)','correct_altitude',  'slope']]

      target = df['Average_fuel_consumption_rate']
      return features.values, target.values
    else:
      return None, None


# Function to slice data into chunks and append to lists
def slice_and_append(features, target, sequence_length):
    num_chunks = len(features) // sequence_length
    X_list, y_list = [], []

    for i in range(num_chunks):
        start_idx = i * sequence_length
        end_idx = start_idx + sequence_length
        sliced_features = features[start_idx:end_idx]
        sliced_target = target[start_idx:end_idx]
        X_list.append(sliced_features)
        y_list.append(sliced_target)

    return X_list, y_list

# Helper function to process multiple files
def process_and_slice(file_path):
    features, target = process_file(file_path)
    if features is not None and target is not None:
        return slice_and_append(features, target, SEQUENCE_LENGTH)
    return [], []

# Function to process a folder in parallel with tqdm for progress tracking
def process_folder_parallel(folder_path, num_workers=4):
    all_X, all_y = [], []
    file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Use tqdm to show progress of file processing
    with Pool(num_workers) as p:
        for X_list, y_list in tqdm(p.imap(process_and_slice, file_paths), total=len(file_paths), desc="Processing files"):
            all_X.extend(X_list)
            all_y.extend(y_list)

    return all_X, all_y

# Main processing logic
def main():
    X_augmented = []
    y_augmented = []

    # Process each folder for data augmentation

    folder_path = '/content/513_aug_3slices_rate'
    X_aug, y_aug = process_folder_parallel(folder_path, num_workers=4)
    X_augmented.extend(X_aug)
    y_augmented.extend(y_aug)

    return X_augmented, y_augmented

# Example call to main

X_augmented, y_augmented = main()






Processing files: 100%|██████████| 35769/35769 [13:30<00:00, 44.11it/s]


In [3]:

# Function to pad or truncate sequences
def pad_or_truncate(sequence, length):
    if len(sequence) > length:
        return sequence[:length]
    elif len(sequence) < length:
        return np.pad(sequence, ((0, length - len(sequence)), (0, 0)), mode='constant')
    else:
        return sequence



In [4]:
X_original = []
y_original = []
# Apply padding/truncating to ensure consistent sequence length
X_original = [pad_or_truncate(x, SEQUENCE_LENGTH) for x in X_original]
y_original = [pad_or_truncate(y.reshape(-1, 1), SEQUENCE_LENGTH) for y in y_original]
X_augmented = [pad_or_truncate(x, SEQUENCE_LENGTH) for x in X_augmented]
y_augmented = [pad_or_truncate(y.reshape(-1, 1), SEQUENCE_LENGTH) for y in y_augmented]


In [5]:
np.array(X_augmented).shape

(67245, 600, 10)

In [ ]:
np.array(y_augmented).shape

In [ ]:
np.array(X_augmented).shape

In [ ]:
np.array(X_augmented).shape

In [6]:
import numpy as np

# Convert lists to numpy arrays if they aren't already
X_original = np.array(X_original)
y_original = np.array(y_original)
X_augmented = np.array(X_augmented)
y_augmented = np.array(y_augmented)


# Saving to npz file
np.savez(f'/content/drive/MyDrive/513_{SEQUENCE_LENGTH}_3slices_shift_rate.npz', X_original=X_original, y_original=y_original, X_augmented=X_augmented, y_augmented=y_augmented)
